In [38]:
# Task 1: Data Loading and Initial Exploration

import pandas as pd
import matplotlib.pyplot as plt

titanic_data = pd.read_excel("/Users/clairestewart/ML-fundamentals-2025/titanic3.xls")

# Statistical description of the data to see trends in the dataset 
print(titanic_data.describe())

# need to figure out ways to visualize data 

            pclass     survived          age        sibsp        parch  \
count  1309.000000  1309.000000  1046.000000  1309.000000  1309.000000   
mean      2.294882     0.381971    29.881135     0.498854     0.385027   
std       0.837836     0.486055    14.413500     1.041658     0.865560   
min       1.000000     0.000000     0.166700     0.000000     0.000000   
25%       2.000000     0.000000    21.000000     0.000000     0.000000   
50%       3.000000     0.000000    28.000000     0.000000     0.000000   
75%       3.000000     1.000000    39.000000     1.000000     0.000000   
max       3.000000     1.000000    80.000000     8.000000     9.000000   

              fare        body  
count  1308.000000  121.000000  
mean     33.295479  160.809917  
std      51.758668   97.696922  
min       0.000000    1.000000  
25%       7.895800   72.000000  
50%      14.454200  155.000000  
75%      31.275000  256.000000  
max     512.329200  328.000000  


In [119]:
# Task 2: Managing Missing Values

missing_values = titanic_data.isnull()
print(missing_values.sum())


# Before any data is changed the columns with missing data are: age, fare, cabin, embarked, boat, body, and home.dest 

# Age:
# Filling in the missing values with the mean age. Median is similar to mean, so either would work. 
mean_age = titanic_data['age'].mean()
titanic_data['age'] = titanic_data['age'].fillna(mean_age, inplace=True)

# Fare
# Filling in fares with the mode 
mode_fare =  titanic_data['fare'].mode() 
#titanic_data['fare'].fillna(mode_fare, inplace=True)

# Cabin
mode_cabin = titanic_data['cabin'].mode()
#titanic_data['cabin'].fillna(mode_cabin, inplace=True)

# Home.Dest
home = titanic_data['home.dest'].mode()
titanic_data['home.dest'] = titanic_data['home.dest'].fillna(home)


pclass          0
survived        0
name            0
sex             0
age          1309
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        2
boat          823
body         1188
home.dest     564
dtype: int64


/var/folders/0z/v55pd5lx53bf4cg9q09ppwy00000gn/T/ipykernel_13374/1952591486.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  titanic_data['age'].fillna(mean_age, inplace=True)
